---
# Intertrial Variability: 1. Preprocessing

In [1]:
import sys; sys.path.insert(1, '../')
from scripts.preproc import load_prepare_data, epoch_clean
import os
import pickle as pkl
from scripts.util import custom_logger, concatenate_epochs

c:\Users\lukas\Dokumente\projects\variabilityanalysis-paper\notebooks\..\scripts\classes\classes.py:22: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,


---
# Define Preprocessing Parameters

In [2]:
# Paths and Names
data_pth = 'D:/data/Psychiatrie_Autismus_2012/'
folders = ['Neurotypicals', 'Asperger']
file_codes = ['PL', 'PS']
group_name = ['control', 'asd']

# Trigger-specific Variables
triglens = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 26, 27, 70]
triglabels = [[10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 30], 
              [40, 40, 40, 40, 40, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 60]]
stimuli = [['SF', 'LR', 'GPL'], ['SR', 'LF', 'GPS']]  # Codes {S: Small, L: Large, F: Frequent, R: Rare}
round_to = 1
onoff = [3, 3]

# Preprocessing Parameters
baseline = None  # No Baseline Correction
trial_time = (-0.2, 0.5)  # Trial time span specification
filtfreq = [1, 30]  # Band-pass frequencies
rereference = ['TP9', 'TP10']  # Reference Channels
art_thresh = None  # Artifact Threshold
srate = 500  # Sampling rate to resample to
rm_bad_chans = False  # No custom Bad Channel Detection
use_ransac=True  # use RANSAC Bad Channel Detection
csd = False  # Don't Use Current Source Density
perform_ICA = True  # Calculate ICA and Remove Blink-related Artifacts
n_jobs = -1  # Number of Cores to Utilize During Parallelization

---
## Load, preprocess and epoch data

In [3]:
all_data_raw = {} 
all_data_epoch = {}

subject_list = [[n for n in os.listdir(os.path.join(data_pth,folder)) if not n.startswith('.') 
                if not n.endswith('.txt')] for folder in folders]

save_path = '../processed/data_preprocessed.pkl'

for i in range(len(subject_list)):
    all_data_raw[group_name[i]] = {}
    all_data_epoch[group_name[i]] = {}
    for j in range(len(subject_list[i])):
        subname = subject_list[i][j][0:2]
        all_data_raw[group_name[i]][subname] = {}

        # Get Paths of all condition of current subject
        conds = os.listdir(os.path.join(data_pth, folders[i], subject_list[i][j]))
        conds_of_interest = [k for k in conds if any([k.startswith('PL'), k.startswith('PS')]) 
                            if not k.startswith('.') if k.endswith('.vhdr')]
        filepaths = [os.path.join(data_pth, folders[i], subject_list[i][j], k) for k in conds_of_interest]
        filepaths.sort()
        
        # Initialize Logging
        logger = custom_logger(os.path.dirname(save_path) + '/logs/' + subject_list[i][j] + '.log')
        
        # Raws
        raws = [load_prepare_data(filepath, rereference, filtfreq, perform_ICA=perform_ICA, logger=logger,
                rm_bad_chans=rm_bad_chans, n_jobs=n_jobs) for filepath in filepaths]
        # Epochs
        epocheds = [epoch_clean(raw, baseline, trial_time, onoff, art_thresh, srate, 
                    round_to, triglens, triglabel, logger=logger, stim_names=stimulus, csd=csd,
                    use_ransac=use_ransac) for raw, triglabel, stimulus in zip(raws, triglabels, stimuli)]
        epochs = concatenate_epochs(epocheds)


        all_data_raw[group_name[i]][subname][file_codes[0]] = raws[0]
        all_data_raw[group_name[i]][subname][file_codes[1]] = raws[1]
        all_data_epoch[group_name[i]][subname] = epochs

with open(save_path, 'wb') as f:
    pkl.dump([all_data_raw, all_data_epoch], f)
    print('saved')

c:\Users\lukas\Dokumente\projects\variabilityanalysis-paper\notebooks\..\scripts\preproc\preproc.py:79: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(pth, preload=True, verbose=0)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    2.3s


channel already dropped
Fitting ICA to data using 32 channels (please be patient, this may take a while)


[Parallel(n_jobs=8)]: Done  32 out of  32 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  32 out of  32 | elapsed:    2.5s finished
c:\Users\lukas\Dokumente\projects\variabilityanalysis-paper\notebooks\..\scripts\preproc\preproc.py:136: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components=25, method=method)


Selecting by number: 25 components
Fitting ICA took 4.4s.
Using EOG channel: Fp1
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Now detecting blinks and generating corresponding events
Found 58 significant peaks
Number of EOG events detected: 58
Not setting metadata
Not setting metadata
58 matching events found
No baseline correction applied
Loading data for 58 events and 501 original time points ...
2 bad epochs dropped
Using EOG ch

c:\Users\lukas\Dokumente\projects\variabilityanalysis-paper\notebooks\..\scripts\preproc\preproc.py:79: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(pth, preload=True, verbose=0)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3']
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Interpolating bad channels
    Automatic origin fit: head of radius 92.9 mm
Computing interpolation matrix from 31 sensor positions
Interpolating 1 sensors
Not setting metadata
Not setting metadata
398 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    6.0s
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    6.0s finished


Loading data for 398 events and 351 original time points ...
0 bad epochs dropped
saved
